# 0_entities

### Importando modulos gerais

In [22]:
import os
import sys
import subprocess
from io import StringIO

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import PyPDF2

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import pytesseract

# import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron
import modules.nova_extracao_pdf_pesquisavel as novaextra 
import modules.trata_model as tmod
import modules.trata_pdf as tpdf
import modules.utils as utl


import json
import spacy
from spacy import displacy
import pandas as pd
from spacy.tokens import Span
from spacy.matcher import Matcher
from spacy.tokens import Token

nlp = spacy.load("pt_core_news_sm")


# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"

# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"



matcher = Matcher(nlp.vocab)

### Patterns

In [10]:
matcher = Matcher(nlp.vocab)

In [11]:
#======================================== 1. CABECALHO




# 3. Data e Hora de Emissão:
data_hora_emissao_pattern = [
    {"LOWER": "data"},
    {"LOWER": "e"},
    {"LOWER": "hora"},
    {"LOWER": "da"},
    {"LOWER": "emissão"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"SHAPE": "dd/dd/dddd"},
    {"IS_SPACE": True, "OP": "*"},
    {"SHAPE": "dd:dd:dd"}
]
matcher.add("DATA_HORA_EMISSAO", [data_hora_emissao_pattern])


# 4. Código de Verificação:
codigo_verificacao_pattern = [
    {"LOWER": "código"},
    {"LOWER": "verificação"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_ASCII": True, "LENGTH": 9}
]
matcher.add("CODIGO_VERIFICACAO", [codigo_verificacao_pattern])




#========================================  5. VALOR TOTAL
valor_total_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "total"},
    {"LOWER": "da", "OP": "?"},
    {"LOWER": "nota", "OP": "?"},
    {"TEXT": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_TOTAL", [valor_total_pattern])





#======================================== 7. VALORES E IMPOSTOS
# 1. VALOR_SERVICOS
valor_servicos_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "serviços"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]

matcher.add("VALOR_SERVICOS", [valor_servicos_pattern])


# 2. VALOR DEDUÇÃO:
valor_deducao_pattern = [
    {"LOWER": "dedução"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]

matcher.add("VALOR_DEDUCAO", [valor_deducao_pattern])



# 3. DESC. INCOND: RASTER_PDF
valor_incondR_pattern = [
    {"LOWER": "base"},
    {"LOWER": "de"},
    {"IS_SPACE": True},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"ORTH": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}    
]
matcher.add("VALOR_INCONDP", [valor_incondR_pattern])


# 3.A DESC. INCOND: - PDF_Pesquisavel   #DESC. INCOND:
valor_incond_patternP = [
    {"LOWER": "desc"},
    {"IS_PUNCT": True, "OP": "?"},
    {"LOWER": "incond"},
    {"ORTH": ":"},
    {"IS_SPACE": True},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"ORTH": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}    
]
matcher.add("VALOR_INCONDR", [valor_incond_patternP])



# 4. BASE DE CÁLCULO:  RASTER_PDF
valor_calculoR_pattern = [
    {"LOWER": "cálculo"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_CALCULOR", [valor_calculoR_pattern])


# 4.A BASE DE CÁLCULO:  PDF_P
valor_calculoP_pattern = [
    {"LOWER": "base"},
    {"LOWER": "de"},
    {"LOWER": "cálculo"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_CALCULOP", [valor_calculoP_pattern])



# 5. Alíquota d,dd
valor_aliquota_pattern = [
    {"LOWER": "alíquota"},
    {"ORTH": ":"},
    {"SHAPE": "d,dd", "OP": "?"},
    {"ORTH": "%"}

]
matcher.add("VALOR_ALIQUOTA", [valor_aliquota_pattern])

# 5.1 Alíquota d
valor_aliquota2_pattern = [
    {"LOWER": "alíquota"},
    {"ORTH": ":"},
    {"SHAPE": "d", "OP": "?"},
    {"ORTH": "%"}

]
matcher.add("VALOR_ALIQUOTA2", [valor_aliquota2_pattern])



# 6. VALOR ISS:
valor_iss_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "iss"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_ISS", [valor_iss_pattern])



# 7. VALOR ISS RETIDO:
valor_issretido_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "iss"},
    {"LOWER": "retido"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_ISSRETIDO", [valor_issretido_pattern])


# 8. DESC. COND:
valor_desccond_pattern = [
    {"LOWER": "desc"},
    {"ORTH": "."},
    {"LOWER": "cond"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_DESCCOND", [valor_desccond_pattern])


# 9. VALOR PIS:
valor_pis_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "pis"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_PIS", [valor_pis_pattern])


# 10. VALOR COFINS:
valor_cofins_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "cofins"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_COFINS", [valor_cofins_pattern])




# 11. VALOR IR:
valor_ir_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "ir"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_IR", [valor_ir_pattern])


# 12. VALOR INSS:
valor_inss_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "inss"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_INSS", [valor_inss_pattern])


# 13. VALOR CSLL:
valor_csll_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "csll"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_CSLL", [valor_csll_pattern])



# 14. OUTRAS RETENÇÕES:
valor_outrasreten_pattern = [
    {"LOWER": "outras"},
    {"LOWER": "retenções"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_OUTRAS", [valor_outrasreten_pattern])



# 15. VALOR LÍQUIDO:
valor_liquido_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "líquido"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_LIQUIDO", [valor_liquido_pattern])





#======================================== 9. OUTRAS INFORMAÇOES / CRITICAS
# 1. EXIGIBILIDADE ISS
exigibilidade_iss_pattern = [
    {"LOWER": "exigibilidade"},
    {"LOWER": "iss"},
    {"LOWER": {"IN": ["exigivel", "não exigivel"]}}
]
matcher.add("EXIGIBILIDADE_ISS", [exigibilidade_iss_pattern])


# 2. REGIME TRIBUTAÇÃO
padrao_regime_tributacao = [
    {"LOWER": "regime"},
    {"LOWER": "tributação"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_ALPHA": True, "OP": "+"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_ALPHA": True, "OP": "*"},
    {"IS_SPACE": True, "OP": "*"},
    {"LOWER": "simples", "OP": "?"},
    {"IS_ALPHA": True, "OP": "*"}
]
matcher.add("REGIME_TRIBUTACAO", [padrao_regime_tributacao])

# 3. SIMPLES NACIONAL = NAO
simples_nacional_nao_pattern = [
    {"LOWER": "simples"},
    {"LOWER": "nacional"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"LOWER": "não"}
]
matcher.add("SIMPLES_NACIONAL_NAO", [simples_nacional_nao_pattern])

# 3.1 SIMPLES NACIONAL = SIM
simples_nacional_pattern = [
    {"LOWER": "simples"},
    {"LOWER": "nacional"},
    {"IS_SPACE": True, "OP": "*"},
    {"LOWER": "sim", "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"ORTH": "(", "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_DIGIT": True, "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"ORTH": ",", "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_DIGIT": True, "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"ORTH": "%", "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"ORTH": ")", "OP": "?"}
]
matcher.add("SIMPLES_NACIONAL_SIM", [simples_nacional_pattern])


# 4. ISSQN RETIDO
issqn_retido_pattern = [
    {"LOWER": "issqn"},
    {"LOWER": "retido"},
    {"IS_SPACE": True, "OP": "*"},
    {"LOWER": {"IN": ["sim", "não"]}}
]
matcher.add("ISSQN_RETIDO", [issqn_retido_pattern])


# 5. LOCAL. PRESTAÇÃO SERVIÇO
local_prestacao_servico_pattern = [
    {"LOWER": "local"},
    {"ORTH": "."},
    {"LOWER": "prestação"},
    {"LOWER": "serviço"},
    {"IS_SPACE": True, "OP": "+"},  # para lidar com múltiplos espaços
    {"IS_ALPHA": True, "OP": "+"},  # para a cidade
    {"ORTH": "-", "OP": "?"},
    {"IS_UPPER": True, "LENGTH": 2, "OP": "?"}  # para a sigla do estado
]
matcher.add("LOCAL_PRESTACAO_SERVICO", [local_prestacao_servico_pattern])

# 6. LOCAL INCIDÊNCIA
local_incidencia_pattern = [
    {"LOWER": "local"},
    {"IS_PUNCT": True, "OP": "?"},
    {"LOWER": "incidência"},
    {"IS_ALPHA": True, "OP": "+"},  # Nome da cidade
    {"ORTH": "-", "OP": "?"},  # Hífen opcional
    {"SHAPE": "XX", "OP": "?"}  # Sigla do estado
]
matcher.add("LOCAL_INCIDENCIA", [local_incidencia_pattern])


# observacao_pattern = [
#     {"LOWER": "observação"},
#     {"ORTH": ":"},
#     {"IS_SPACE": True, "OP": "*"},
#     {"LOWER": "-", "OP": "?"},
#     {"IS_PRINT": True, "OP": "+"}
# ]

# matcher.add("OBSERVACAO", [observacao_pattern])# 6. Alíquota
valor_aliquota_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "iss"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_DIGIT": True, "OP": "+"},
    {"ORTH": "", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"},
    {"ORTH": "%"}
]


### Raster PDF

In [5]:
def run_ocrmypdf(input_file, output_file):
    command = [
        'ocrmypdf',
        '--language', 'por',
        '--deskew',
        input_file,
        output_file
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if result.returncode == 0:
        print(f"OCRmyPDF completed successfully. Output saved to {output_file}.")
    else:
        print(f"OCRmyPDF failed with error: {result.stderr.decode('utf-8')}")

In [6]:
input_file = "/home/dani-boy/extractNF/pipeline_extracao_documentos/0_arquivos_teste_pipeline/pdf_raster/Mage/nf_689__pmmacae__8a_med_co_22_22_tb_contrato_045_22_1.pdf"
output_file = "/home/dani-boy/extractNF/processamentos/temp/documento.pdf"

In [7]:
# 1. XXX Executar o comando OCRmyPDF
run_ocrmypdf(input_file, output_file)

# 2. XXX Executar o comando OCRmyPDF    
!pdftotext processamentos/temp/documento.pdf processamentos/temp/txt/documento.txt

# 3. XXX Ler o arquivo TXT
with open('processamentos/temp/txt/documento.txt', 'r', encoding='utf-8') as arquivo:
    texto_OCR_R = arquivo.read()

texto_PDF_Raster = texto_OCR_R.replace('\n', ' ')
texto_PDF_Raster

OCRmyPDF completed successfully. Output saved to /home/dani-boy/extractNF/processamentos/temp/documento.pdf.


'02/08/23, 16:33  Nota Fiscal de Serviços Eletrônica (NFSe) Número da Nota:  PREFEITURA MUNICIPAL DE MAGE  ia  Competência:  SECRETARIA MUNICIPAL DA FAZENDA  Agosto/2023  NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e  Data e Hora da Emissão: 02/08/2023 16:29:00  Código Verificação:  OCDD6FB51  PRESTADOR DE SERVIÇOS CPF/CNPJ:  Inscrição Municipal:  Telefone:  Inscrição Estadual:  33.462.862/0001-95  li:  4337  al  Nome/Razão Social: SÃO MARCOS TERRAPLENAGEM E CONSTRUÇÃO LTDA. Nome de Fantasia: Endereço: RUA DAS MARGARIDAS ,578 ,SANTA DALILA - Magé-RJ E-mail:  TOMADOR DE SERVIÇOS CPF/CNPJ: 29.115.474/0001-60 HINSC:MUNICIPAL:  RG:  |  Telefone:  Inscrição Estadual:  Nome/Razão Social: MUNICÍPIO DE MACAE Endereço: ; PRESIDENTE FELICIANO SODRE Nº 534 PAÇO E-mail: NÃO INFORMADO  MUNICIPAL  BAIRRO:  CENTRO  CIDADE:  MACAE  ;  - RJ CEP: 27913080  DISCRIMINAÇÃO DOS SERVIÇOS Serviço de Recomposição do Pavimento Asfáltico (Tapa Buraco), na cidade de Macaé/RJ, abrangendo Perímetro Urbano, Região Serr

In [ ]:
"SHAPE": "dd.ddd.ddd/dddd-dd"

In [35]:
from spacy.language import Language

# Função para definir o atributo "is_cnpj"
@Language.component("set_cnpj_attribute")
def set_cnpj_attribute(doc):
    for i, token in enumerate(doc):
        if i < len(doc) - 1:
            next_token = doc[i + 1]
            if token.shape_ == "dd.ddd.ddd/" and next_token.shape_ == "dddd-dd":
                token._.is_cnpj = True
                next_token._.is_cnpj = True
            else:
                token._.is_cnpj = False
    return doc

In [24]:
# Registro do atributo 'is_cnpj'
Token.set_extension('is_cnpj', default=False)

In [25]:
# Inicialização do Matcher
matcher = Matcher(nlp.vocab)

In [30]:
# Padrão para encontrar CNPJ
pattern = [
    {"SHAPE": "dd.ddd.ddd/"},
    {"SHAPE": "dddd-dd"}
]
# Adição do padrão ao Matcher
matcher.add("CNPJ_PATTERN", [pattern])

In [38]:
# Função para aplicar o matcher
@Language.component("apply_cnpj_matcher")
def apply_cnpj_matcher(doc):
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        for token in span:
            token._.is_cnpj = True
    return doc

In [36]:
nlp.add_pipe("set_cnpj_attribute") # Adicione esta etapa se você quiser definir o atributo manualmente

<function __main__.set_cnpj_attribute(doc)>

In [40]:
nlp.add_pipe("apply_cnpj_matcher")  # Adicione esta etapa para aplicar o matcher

<function __main__.apply_cnpj_matcher(doc)>

In [45]:
doc = nlp(texto_PDF_Raster)

In [ ]:
token.idx

In [48]:
for token in doc:
    if token._.is_cnpj:
        print(f'{token.idx:>5} | token.text: {token.text}, token.shape: {token.shape_}, token._.is_cnpj: {token._.is_cnpj} ') 
    #print(token.text, token._.is_cnpj)

  378 | token.text: 33.462.862/, token.shape: dd.ddd.ddd/, token._.is_cnpj: True 
  389 | token.text: 0001-95, token.shape: dddd-dd, token._.is_cnpj: True 
  591 | token.text: 29.115.474/, token.shape: dd.ddd.ddd/, token._.is_cnpj: True 
  602 | token.text: 0001-60, token.shape: dddd-dd, token._.is_cnpj: True 
 2233 | token.text: 29.115.474/, token.shape: dd.ddd.ddd/, token._.is_cnpj: True 
 2244 | token.text: 0001-60, token.shape: dddd-dd, token._.is_cnpj: True 


In [52]:
cnpjs = []  # Lista para armazenar os CNPJs completos
indices = []  # Lista para armazenar os índices dos tokens que são CNPJs

for token in doc:
    if token._.is_cnpj:
        indices.append(token.i)  # Adicione o índice do token à lista

# Agrupe as partes do CNPJ
for i in range(0, len(indices), 2):  # Vai de dois em dois
    try:
        first_part = doc[indices[i]].text  # Primeira parte do CNPJ
        second_part = doc[indices[i + 1]].text  # Segunda parte do CNPJ
        full_cnpj = first_part + second_part  # Concatena as duas partes
        cnpjs.append(full_cnpj)
    except IndexError:
        print("Índice fora dos limites. Talvez o CNPJ esteja incompleto.")
        
print(cnpjs)



['33.462.862/0001-95', '29.115.474/0001-60', '29.115.474/0001-60']


In [ ]:








# Exemplo de uso
# Suponha que 'nlp' é o seu objeto de linguagem spaCy já carregado
# nlp.add_pipe(set_cnpj_attribute)  # Adicione esta etapa se você quiser definir o atributo manualmente
# 

# doc = nlp("O CNPJ da empresa é 33.462.862/0001-95.")
# for token in doc:
#     print(token.text, token._.is_cnpj)

In [20]:
matcher = Matcher(nlp.vocab)


# 1. CPF/CNPJ:
valor_cnpjprestadorR_pattern = [
    {"LOWER": "inscricao"},
    {"LOWER": "estadual"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"SHAPE": "dd.ddd.ddd/", "OP": "+"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"SHAPE": "dddd-dd", "OP": "*"}

]
matcher.add("VALOR_CPF_CNPJ:", [valor_cnpjprestadorR_pattern])

In [21]:
doc = nlp(texto_PDF_Raster)

# Executar o Matcher no Doc
matches = matcher(doc)

# Exibir os resultados
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Obter a string de identificação
    span = doc[start:end]  # Obter o trecho correspondente
    print(f'\nstring_id: {string_id:>30}   span.text:{span.text:>45} ')

### PDF Pesquisavel

In [ ]:
# PDF Pesquisavel

file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_23/MESQUITA_PDF_31282023_2258/159871/2023 -8.pdf"


pdf_document = fitz.open(file_path)


page_number = 0  # Defina o número da página que deseja analisar
page = pdf_document[page_number]

# Extrair texto dentro do retângulo
text_P = page.get_text("text")


pdf_document.close()

texto_PDF_Pesquisavel = text_P.replace('\n', ' ')
texto_PDF_Pesquisavel 


In [ ]:
doc = nlp(texto_PDF_Pesquisavel)

# Executar o Matcher no Doc
matches = matcher(doc)

# Exibir os resultados
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Obter a string de identificação
    span = doc[start:end]  # Obter o trecho correspondente
    print(f'\nstring_id: {string_id:>30}   span.text:{span.text:>45} ')

### Amostra

In [15]:
#texto_amostra = "ALÍQUOTA: 2,01% VALOR ISS"

texto_amostra = " Telefone:  Inscrição Estadual:  33.462.862/0001-95  li:  4337 "

doc = nlp(texto_amostra)
matches = matcher(doc)

# Exibir os resultados
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Obter a string de identificação
    span = doc[start:end]  # Obter o trecho correspondente
    print(f"{string_id}: {span.text}")

In [ ]:
def find_cnpj(df):
    cnpj_list = []
    for i in range(len(df) - 1):
        if df.loc[i, 'T_shape'] == 'dd.ddd.ddd/' and df.loc[i + 1, 'T_shape'] == 'dddd-dd':
            cnpj = df.loc[i, 'T_texto'] + df.loc[i + 1, 'T_texto']
            cnpj_list.append(cnpj)
    return cnpj_list


In [ ]:
# Suponhamos que 'df' é o seu DataFrame
cnpjs = find_cnpj(df)
print("CNPJs encontrados:", cnpjs)

In [16]:
# Analisys
syntatic = pd.DataFrame(data=[], \
  columns=["id", "T_texto","T_shape", "T_is_alpha", "T_is_digit", "T_is_title", "T_is_punct", "T_is_sent_start", "T_is_right_punct", "T_is_stop", "T_is_quote", "T_is_currency", "T_morph"])
i = 0
for token in doc:
    syntatic.loc[i,"id"] = token.i
    syntatic.loc[i,"T_texto"] = token.text
    syntatic.loc[i,"T_shape"] = token.shape_
    syntatic.loc[i,"T_is_alpha"] = token.is_alpha
    syntatic.loc[i,"T_is_digit"] = token.is_digit
    syntatic.loc[i,"T_is_title"] = token.is_title
    syntatic.loc[i,"T_is_punct"] = token.is_punct
    syntatic.loc[i,"T_is_sent_start"] = token.is_sent_start
    syntatic.loc[i,"T_is_right_punct"] = token.is_right_punct
    syntatic.loc[i,"T_is_stop"] = token.is_stop
    syntatic.loc[i,"T_is_quote"] = token.is_quote
    syntatic.loc[i,"T_is_currency"] = token.is_currency
    syntatic.loc[i,"T_morph"] = token.morph
    i = i+1

syntatic.head(80)

,id,T_texto,T_shape,T_is_alpha,T_is_digit,T_is_title,T_is_punct,T_is_sent_start,T_is_right_punct,T_is_stop,T_is_quote,T_is_currency,T_morph
0,0,,,False,False,False,False,True,False,False,False,False,()
1,1,Telefone,Xxxxx,True,False,True,False,True,False,False,False,False,"(Gender=Masc, Number=Sing)"
2,2,:,:,False,False,False,True,False,False,False,False,False,()
3,3,,,False,False,False,False,False,False,False,False,False,()
4,4,Inscrição,Xxxxx,True,False,True,False,False,False,False,False,False,"(Gender=Fem, Number=Sing)"
5,5,Estadual,Xxxxx,True,False,True,False,False,False,False,False,False,(Number=Sing)
6,6,:,:,False,False,False,True,False,False,False,False,False,()
7,7,,,False,False,False,False,False,False,False,False,False,()
8,8,33.462.862/,dd.ddd.ddd/,False,False,False,False,True,False,False,False,False,(NumType=Card)
9,9,0001-95,dddd-dd,False,False,False,False,False,False,False,False,False,(NumType=Range)


In [ ]:
syntatic.to_excel("syntatic.xlsx", index=False)

In [ ]:
{"IS_DIGIT": True, "OP": "+"},
{"ORTH": ",", "OP": "?"},
{"IS_DIGIT": True, "OP": "*"},
{"ORTH": "%"}



{"SHAPE": "XX", "OP": "?"}

# {"LOWER": "alíquota"} busca a palavra "alíquota" em qualquer combinação de maiúsculas e minúsculas.
# {"ORTH": ":"} busca o caractere ":".
# {"IS_SPACE": True, "OP": "*"} permite zero ou mais espaços.
# {"IS_DIGIT": True, "OP": "+"} busca um ou mais dígitos.
# {"ORTH": ",", "OP": "?"} busca zero ou um caractere de vírgula (para números fracionários).
# {"IS_DIGIT": True, "OP": "*} busca zero ou mais dígitos (novamente, para números fracionários).
# {"ORTH": "%"} busca o símbolo "%".

In [49]:
# Lemmatization for tokens 
lemmatization = pd.DataFrame(data=[], \
  columns=["id", "Texto","Lemma", "Tag", "Tag_explainned", "token_POS", "POS_explainned", "dep", "T. Head", "dep explained"])
i = 0
for token in doc:
    lemmatization.loc[i,"id"] = token.i
    lemmatization.loc[i,"Texto"] = token.text
    lemmatization.loc[i,"Lemma"] = token.lemma_
    lemmatization.loc[i,"Tag"] = token.tag_
    lemmatization.loc[i,"Tag_explainned"] = spacy.explain(token.tag_)
    lemmatization.loc[i,"token_POS"] = token.pos_
    lemmatization.loc[i,"POS_explainned"] = spacy.explain(token.pos_)
    lemmatization.loc[i,"dep"] = token.dep_
    lemmatization.loc[i,"T. Head"] = token.head.text
    lemmatization.loc[i,"dep explained"] = token.morph
    
    i = i+1
lemmatization

,id,Texto,Lemma,Tag,Tag_explainned,token_POS,POS_explainned,dep,T. Head,dep explained
0,0,02/08/23,02/08/23,NOUN,noun,NOUN,noun,ROOT,02/08/23,"(Gender=Masc, Number=Plur)"
1,1,",",",",PUNCT,punctuation,PUNCT,punctuation,punct,02/08/23,()
2,2,16:33,16:33,PROPN,proper noun,PROPN,proper noun,dep,02/08/23,"(Gender=Masc, Number=Sing)"
3,3,,,SPACE,space,SPACE,space,dep,16:33,()
4,4,Nota,Nota,PROPN,proper noun,PROPN,proper noun,flat:name,16:33,(Number=Sing)
...,...,...,...,...,...,...,...,...,...,...
519,519,IBPT,IBPT,PROPN,proper noun,PROPN,proper noun,appos,Alíq,"(Gender=Masc, Number=Sing)"
520,520,),),PUNCT,punctuation,PUNCT,punctuation,punct,Alíq,()
521,521,,,SPACE,space,SPACE,space,dep,),()
522,522,https://nfs-e.mage.rj.gov.br,https://nfs-e.mage.rj.gov.br,PROPN,proper noun,PROPN,proper noun,appos,Valor,"(Gender=Masc, Number=Sing)"


In [ ]:
lemmatization.to_excel("lemmatization.xlsx", index=False)

In [ ]:
linhas = [linha for linha in texto_OCR.split('\n') if linha.strip()]

### Contas para simulacao

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate


### Funcoes

### Criando Patterns especificos

### Salvando e carregando patterns e modelos de patterns e criacao de logs JSONL

In [ ]:
def write_patterns_to_file(patterns, colors, filename):
    data = {"patterns": patterns, "colors": colors}
    with open(filename, "w") as f:
        json.dump(data, f, ensure_ascii=True, indent=2)
        
        
        
def load_patterns_and_colors(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
        patterns = data["patterns"]
        colors = data["colors"]
    return patterns, colors   


# my_str.encode('utf-8')

In [ ]:
filename = "/home/wklinux/spaCy/configuracoes/pattern_test.json"  



In [ ]:
patterns, colors = load_patterns_and_colors(filename)

In [ ]:
write_patterns_to_file(patterns=patterns, colors=colors, filename=filename)

<h3>Modelo de criaçao de registro JSONL</h3>

In [ ]:
# JSONL 'Total do contrato'

import json
with open('output_daniel.jsonl', 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        summary = f"Summary: 'Total do contrato\n\nSpecific information: Contrato {row['8-nro contrato']} tem a quantidade total de {row['9-qt total contrato']} kilos\n\n###\n\n"
        customer_message = "Bom dia, Eu gostaria de saber o total do meu contrato?\n"
        agent_response = f"Agent: O total do seu contrato {row['8-nro contrato']} é {row['9-qt total contrato']} kilos. Mais alguma informação?\n"
        customer_message2 = "Customer: Não, muito obrigado!\n"
        agent_response2 = "Agent: Perfeito, Se precisar de algo é só chamar.\n"
        json_data = {"prompt": summary + customer_message + agent_response + customer_message2 + agent_response2,"completion": agent_response2}
        
        
        json_string = json.dumps(json_data, ensure_ascii=False)
        print(json_string)
        
        f.write(json_string + '\n')

<h3> Exemplos de implementacao de patterns com analise sintatica</h3>

## Entidades

In [ ]:
filename = "/home/wklinux/spaCy/query_utter.json"  

data = load_json(filename)

### amostras de textos

### Inicio dados da entidade

### Tratando entidades

In [ ]:
# Analisys
syntatic = pd.DataFrame(data=[], \
  columns=["id", "T_texto","T_shape", "T_is_alpha", "T_is_digit", "T_is_title", "T_is_punct", "T_is_sent_start", "T_is_right_punct", "T_is_stop", "T_is_quote", "T_is_currency", "T_morph"])
i = 0
for token in doc:
    syntatic.loc[i,"id"] = token.i
    syntatic.loc[i,"T_texto"] = token.text
    syntatic.loc[i,"T_shape"] = token.shape_
    syntatic.loc[i,"T_is_alpha"] = token.is_alpha
    syntatic.loc[i,"T_is_digit"] = token.is_digit
    syntatic.loc[i,"T_is_title"] = token.is_title
    syntatic.loc[i,"T_is_punct"] = token.is_punct
    syntatic.loc[i,"T_is_sent_start"] = token.is_sent_start
    syntatic.loc[i,"T_is_right_punct"] = token.is_right_punct
    syntatic.loc[i,"T_is_stop"] = token.is_stop
    syntatic.loc[i,"T_is_quote"] = token.is_quote
    syntatic.loc[i,"T_is_currency"] = token.is_currency
    syntatic.loc[i,"T_morph"] = token.morph
    i = i+1

syntatic

In [ ]:
for ent in doc.ents:
    print(f'{ent.start:>2} | {ent.text:>21} | {ent.label_:>12} | {ent.id_:>15}  | {ent.end:>2} || {ent.start_char:>3} | {ent.end_char:>3}')

In [ ]:
[ent.orth_ for ent in doc.ents if ent.label_ == "ENTREGUE"]
label_entregue = [ent.orth_ for ent in doc.ents if ent.label_ == "ENTREGUE"]

In [ ]:
token = doc[22]  
print(token.morph, token.text) 
print(token.morph.get("PronType"))  

In [ ]:
# Lemmatization for tokens 
lemmatization = pd.DataFrame(data=[], \
  columns=["id", "Texto","Lemma", "Tag", "Tag_explainned", "token_POS", "POS_explainned", "dep", "T. Head", "dep explained"])
i = 0
for token in doc:
    lemmatization.loc[i,"id"] = token.i
    lemmatization.loc[i,"Texto"] = token.text
    lemmatization.loc[i,"Lemma"] = token.lemma_
    lemmatization.loc[i,"Tag"] = token.tag_
    lemmatization.loc[i,"Tag_explainned"] = spacy.explain(token.tag_)
    lemmatization.loc[i,"token_POS"] = token.pos_
    lemmatization.loc[i,"POS_explainned"] = spacy.explain(token.pos_)
    lemmatization.loc[i,"dep"] = token.dep_
    lemmatization.loc[i,"T. Head"] = token.head.text
    lemmatization.loc[i,"dep explained"] = token.morph
    
    i = i+1
lemmatization

In [ ]:
token_root = [token.text for token in doc if token.dep_ == "ROOT"][0]
token_root

## noun_chunks

In [ ]:
chunks = list(doc.noun_chunks)
for i in range(len(chunks)):
    print(i, chunks[i])

In [ ]:
# VISUALIAZACAO ESTRUTURA DEP, HEAD, ROOT, LEMMA
chunks_valor = []


for chunk in doc.noun_chunks:
  chunk_text = chunk.text
  chunks_valor.append(chunk_text)
  chunk_root = chunk.root.text
  chunk_root_dep = chunk.root.dep_
  chunk_root_head = chunk.root.head.text
  chunk_root_head_dep = chunk.root.head.dep_
  chunk_root_head_lemma = chunk.root.head.lemma_
  chunk_ents = chunk.ents
  chunk_root_ent_type = chunk.root.ent_type_
  
  print(f'1.chunk.text: {chunk_text:>26} | 2.ch.root: {chunk_root:>10} | 3.chunk.root.dep_: {chunk_root_dep:>15} | 4.ch.root.head: {chunk_root_head:>12} | 5.ch.root.head.dep_: {chunk_root_head_dep:>10} |  6.chunk.root.head.lemma_: {chunk_root_head_lemma:>9}')

In [ ]:
displacy.render(doc, style='dep',
                jupyter=True, options={'distance': 120})

In [ ]:
def relevant_chunk(doc, param, i):
    for chunk in doc.noun_chunks:
        chunk_root_head_lemma_ = (chunk.root.head.lemma_).lower()
        if chunk_root_head_lemma_ in ["gostaria", "entregar", "entregueir"]:
            if chunk.root.dep_ == param:
                return chunk.text, i, chunk.doc
            # else:
            #     return "erro", chunk.root.dep_
        elif chunk_root_head_lemma_ in ["qual", "entregar"]:
            if chunk.root.dep_ == "nsubj:pass":
                return chunk.text, i, chunk.doc
        elif chunk_root_head_lemma_ in ["entregar", "entregueir"]:
            if chunk.root.dep_ == "obj":
                return chunk.text, i, chunk.doc   

In [ ]:
chunk_text = [chunk.text for chunk in doc.noun_chunks][0]
chunk_text

In [ ]:
chunk_root_dep = [chunk.root.dep_ for chunk in doc.noun_chunks][0]
chunk_root_dep

In [ ]:
chunk_root_head_lemma = [chunk.root.head.lemma_ for chunk in doc.noun_chunks][0].lower()
chunk_root_head_lemma

In [ ]:
i = 0
for i in range(len(data)):
    doc_query = nlp(data[i]["user_utter"])
    result = relevant_chunk(doc=doc_query, param=chunk_root_dep, i=i)
    # print(result)
    if result:
        bot_utter = nlp(data[i]["bot_utter"])
        user_keys = nlp(data[i]["user_keys"])
        user_desire = nlp(data[i]["user_des"])
        missed_keys = nlp(data[i]["miss_key"])      
        print(bot_utter)
        #print(data[i]["user_utter"])

In [ ]:
# POS Tagging
pos_tagging = pd.DataFrame(data=[], \
  columns=["id", "T_texto","T_lemma_", "T_pos_", "T_tag_", "T_dep_", "T_head", "T_is_sent_start", "T_shape_", "T_is_alpha", "T_is_stop"])
i = 0
for token in doc:
    pos_tagging.loc[i,"id"] = token.i
    pos_tagging.loc[i,"T_texto"] = token.text
    pos_tagging.loc[i,"T_lemma_"] = token.lemma_
    pos_tagging.loc[i,"T_pos_"] = token.pos_
    pos_tagging.loc[i,"T_tag_"] = token.tag_
    pos_tagging.loc[i,"T_dep_"] = token.dep_
    pos_tagging.loc[i,"T_head"] = token.head
    pos_tagging.loc[i,"T_is_sent_start"] = token.is_sent_start
    pos_tagging.loc[i,"T_shape_"] = token.shape_
    pos_tagging.loc[i,"T_is_alpha"] = token.is_alpha
    pos_tagging.loc[i,"T_is_stop"] = token.is_stop

    i = i+1

pos_tagging

## Outras tratativas

In [ ]:
/etc/odbcinst.ini 

In [ ]:
def trata_contrato(doc, n_contrato):
    #nlp = spacy.blank("pt")
    #ruler = nlp.add_pipe("entity_ruler")
    # ruler.add_patterns(patterns)
    # doc = nlp(text)
    
    tokens_contrato = []
    ents_contrato = []
    
    
    for ent in doc.ents:
        span = doc.char_span(ent.start_char, ent.end_char, label=ent.label_)
        ents_contrato.append(span)
        
    for token in doc:
        start = token.idx
        end = start + len(token)
        if token.shape_ == "dddX":
            n_contrato.append(token.text)
        tokens_contrato.append((token.text, start, end))
        
    return doc, tokens_contrato, ents_contrato, n_contrato

In [ ]:
nros_contratos = []
n_contrato = []
for ent in doc.ents:
    if ent.label_ == "CONTRATO":
        ent_texto = nlp(ent.text)
        print(ent_texto)
        doc, tokens_contrato, ents_contrato, n_contrato = trata_contrato(ent_texto, n_contrato)
        
        nros_contratos.append(ent.text)
        
        print(f'{ent.start:>2} | {ent.text:>21} | {ent.label_:>8} | {ent.id_:>15} | {ent.end:>2} || {ent.start_char:>3} | {ent.end_char:>3}')
        
        

In [ ]:
contrato_1 = nros_contratos[0]
contrato_2 = nros_contratos[1]

In [ ]:
novo_doc = nlp(contrato_1)

In [ ]:
trata_contrato(novo_doc)

In [ ]:
doc = nlp("Mr. Best flew to New York on Saturday morning.")
span = doc[0:6]
ents = list(span.ents)
assert ents[0].label == 346
assert ents[0].label_ == "PERSON"
assert ents[0].text == "Mr. Best"

### Saving Patterns

In [ ]:
doc = nlp("I live in NewYork")
with doc.retokenize() as retokenizer:
    heads = [(doc[3], 1), doc[2]]
    attrs = {"POS": ["PROPN", "PROPN"],
             "DEP": ["pobj", "compound"]}
    retokenizer.split(doc[3], ["New", "York"], heads=heads, attrs=attrs)

In [ ]:
for contrato in range(len(list(nros_contratos))):
    doc, tokens, ents = show_ent_new(contrato, patterns=patterns) 

In [ ]:
nros_contratos = []
for ent in doc.ents:
    if ent.label_ == "CONTRATO":
        ent_texto = ent.text
        doc_contrato = nlp(ent_texto)
        print(doc_contrato)
        print()
        for token in doc_contrato:
            #print(token.i, token.text, token.shape_)
            print()
            print()
            qtd_contratops = token.morph.get("Number")
            if "Plur" in qtd_contratops:
                # print("               Ele esta falando mais de um contrato")
                if token.shape_ == "dddX":
                    nro_contrato = token.text
                    nros_contratos.append(nro_contrato)
                    print(f'         Ele esta falando de contratos (plural),  esse e o nro do contrato: {nro_contrato}\n\n')
                else:
                    print(f'         Ele esta falando de contratos (plural) -  Nao ha nro de contrato\n')
                    nros_contratos.append(nro_contrato)
            else:
                # print("               ele esta falando no singular - contrato")    
                if token.shape_ == "dddX":
                    nro_contrato = token.text
                    nros_contratos.append(nro_contrato)
                    print(f'          Ele esta falando de contrato (singular),  esse e o nro do contrato: {nro_contrato}\n\n')
                else:
                    print(f'          Ele esta falando de contrato (singular) - Nao ha nro de contrato\n')
                    nros_contratos.append(nro_contrato)
                    
            print()        
       
# print(nros_contratos)       

In [ ]:
nros_contratos = []
entidades = list(doc.ents)

In [ ]:
len(entidades)

In [ ]:
for i in range(len(entidades)):
    print(f'1. idx: {i} | {doc[i].text:>19} | type: {doc[i].ent_type_:>10} {doc[i].ent_id_} ')

In [ ]:
entidades

In [ ]:
chunks = list(doc.noun_chunks)
for i in range(len(chunks)):
    # if chunks[i].root.ent_type_ == "CONTRATO":
        print(f'1. idx: {i} | {chunks[i].text:>19} | {chunks[i].start:>2} {chunks[i].end:>2} | {chunks[i].start_char:>3}  {chunks[i].end_char:>3} || type: {chunks[i].root.ent_type_:>10} {chunks[i].ents} ')

In [ ]:
[ent for ent in doc.ents if ent.label_ == "CONTRATO"]

In [ ]:
[ent for ent in doc.ents if ent.label_ == "NR_CONTRATO"]

In [ ]:
len([ent for ent in doc.ents if ent.label_ == "CONTRATO"])

In [ ]:
chunks = list(doc.noun_chunks)
for i in range(len(chunks)):
    if chunks[i].root.ent_type_ == "CONTRATO":
        print(f'1. idx: {i} | {chunks[i].text:>19} | {chunks[i].start:>2} {chunks[i].end:>2} | {chunks[i].start_char:>3}  {chunks[i].end_char:>3} || type: {chunks[i].root.ent_type_:>10} {chunks[i].ents} ')
        

In [ ]:
nros_contratos = []
chunks = list(doc.noun_chunks)
for i in range(len(chunks)):
    if chunks[i].root.ent_type_ == "CONTRATO":
        print(f'1. idx: {i} | {chunks[i].text:>19} | {chunks[i].start:>2} {chunks[i].end:>2} | {chunks[i].start_char:>3}  {chunks[i].end_char:>3} || type: {chunks[i].root.ent_type_:>10} {chunks[i].ents} ')
        print()
        doc_contrato = nlp(chunks[i].text)
        for token in doc_contrato:
            #print(token.i, token.text, token.shape_)
            print()
            print()
            qtd_contratops = token.morph.get("Number")
            if "Plur" in qtd_contratops:
                # print("               Ele esta falando mais de um contrato")
                if token.shape_ == "dddX":
                    nro_contrato = token.text
                    nros_contratos.append(nro_contrato)
                    print(f'         Ele esta falando de contratos (plural),  esse e o nro do contrato: {nro_contrato}\n\n')
                else:
                    print(f'         Ele esta falando de contratos (plural) -  Nao ha nro de contrato\n')
            else:
                # print("               ele esta falando no singular - contrato")    
                if token.shape_ == "dddX":
                    nro_contrato = token.text
                    nros_contratos.append(nro_contrato)
                    print(f'          Ele esta falando de contrato (singular),  esse e o nro do contrato: {nro_contrato}\n\n')
                else:
                    print(f'          Ele esta falando de contrato (singular) - Nao ha nro de contrato\n')
            print()        
       
print(nros_contratos)       

In [ ]:
for ent in doc.ents:
    if ent.label_ == "CONTRATO":
        doc_contrato = nlp(ent.text)
        # print(token.text, token.ent_type_, token.shape_)
        qtd_contratops = ent.orth_
        #print(doc_contrato)
        for token in doc_contrato:
            print(token.i, token.text, token.shape_)
            print()
            print()
            qtd_contratops = token.morph.get("Number")
            if "Plur" in qtd_contratops:
                print("Ele esta falando mais de um contrato")
                if token.shape_ == "dddX":
                    nro_contrato = token.text
                    print(f'esse e o nro do contrato: {nro_contrato}')
                else:
                    print(f'Nao ha nro de contrato')
            else:
                print("ele esta falando no singular - contrato")    
                if token.shape_ == "dddX":
                    nro_contrato = token.text
                    print(f'esse e o nro do contrato: {nro_contrato}')
                else:
                    print(f'Nao ha nro de contrato')
                    

In [ ]:
for token in doc:
    if token.ent_type_ == "CONTRATO":
        # print(token.text, token.shape_)
        if token.shape_ == "dddX":
            print(token.text)
        # print(token.text, token.shape_)

In [ ]:
contrato = [ent for ent in doc.ents if ent.label_ == "CONTRATO"]

In [ ]:
contrato

In [ ]:
for ent in doc.ents:
    if ent.label_ == "CONTRATO":
        doc_contrato = nlp(ent.text)

In [ ]:
token_root = [token.text for token in doc if token.dep_ == "ROOT"][0]
token_root

In [ ]:
contrato_keys = contrato[0].text.split('')

In [ ]:
[ent for ent in doc.ents if ent.label_ == "CONTRATO"]

In [ ]:
[ent for ent in doc.ents if ent.label_ == "SAFRA"]

In [ ]:
# Tratamento para contratos
for token in doc:
    if token.ent_type_ == "CONTRATO":
        # print(token.text, token.ent_type_, token.shape_)
        qtd_contratops = token.morph.get("Number")
        # print(qtd_contratops)
        if "Plur" in qtd_contratops:
            print("Ele esta falando mais de um contrato")
        else:
             print("Ele esta falando somente de um contrato")   
        if token.shape_ == "dddX":
            nro_contrato = token.text
            print(f'esse e o nro do contrato: {nro_contrato}')
        else:
            print(f'nao ha numero do contrato')
    
    # elif token.ent_type_ == "CONTRATO":    

In [ ]:
user_utterance = text

In [ ]:
{"POS": "PROPN", "LENGTH": {">=": 10}},


[
  {"POS": "ADJ", "OP": "*"},
  {"POS": "NOUN", "OP": "+"}
  {"POS": "PROPN", "OP": "{2}"}
]

In [ ]:
patternsContrato = [
    {
        "label":"CONTRATO",
        "pattern": [
            {"LOWER": "contrato", "OP":"*"},
            {"SHAPE": "dddX", "OP":"*"},
            {"LOWER": "contratos", "OP":"*"},
            {"SHAPE": "dddX", "OP":"*"}
            
        ]
        
    }
    
]

### Tokens de valor

### Tratamento de noun_chunks

In [ ]:
def relevant_chunk(doc, param, i):
    for chunk in doc.noun_chunks:
        chunk_root_head_lemma_ = (chunk.root.head.lemma_).lower()
        if chunk_root_head_lemma_ in ["gostaria", "entregar", "entregueir"]:
            if chunk.root.dep_ == param:
                return chunk.text, i, chunk.doc
            # else:
            #     return "erro", chunk.root.dep_
        elif chunk_root_head_lemma_ in ["qual"]:
            if chunk.root.dep_ == "ROOT":
                return chunk.text, i, chunk.doc

In [ ]:
chunk_text = [chunk.text for chunk in doc.noun_chunks][0]
chunk_text

In [ ]:
chunk_root_dep = [chunk.root.dep_ for chunk in doc.noun_chunks][0]
chunk_root_dep

In [ ]:
chunk_root_head_lemma = [chunk.root.head.lemma_ for chunk in doc.noun_chunks][0].lower()
chunk_root_head_lemma

In [ ]:
chunks = list(doc.noun_chunks)
for i in range(len(chunks)):
    print(f'1. idx: {i} | {chunks[i].text:>13} | {chunks[i].start:>2} {chunks[i].end:>2} | {chunks[i].start_char:>2}  {chunks[i].end_char:>2} || {chunks[i].root.ent_type_:>10} {chunks[i].ents} ')
    
    
    # print(f'1.chunk.text: {chunk_text:>15} id: {chunk.id_} || {chunk.start:>2} | {chunk.end:>2} || {chunk.start_char:>2} | {chunk.end_char:>2} || 4.chunk_root_ent_type {chunk_root_ent_type:>1} | 5.chunk_ents {chunk_ents}')

In [ ]:
text = "Preciso saber o saldo do contrato 342F da fazenda Santa Rita"

doc, tokens, ents = show_ent_new(text, patterns=patterns)
print(doc)
print()
## GARANTIR que a ordem dos Tokens esteja correta (ascendente)
seq_tokens_id = []
seq_tokens_valor = []
chunks_valor = []
## tokens_ids + Lista de todos os Tokens da frase
for token in doc:
    seq_tokens_id.append(token.i)
    
    
displacy.render(doc, style="ent", options={"colors": colors})

displacy.render(doc, style='dep',
                jupyter=True, options={'distance': 120})




for chunk in doc.noun_chunks:
  chunk_text = chunk.text
  chunks_valor.append(chunk_text)
  chunk_root = chunk.root.text
  chunk_root_dep = chunk.root.dep_
  chunk_root_head = chunk.root.head.text
  chunk_root_head_dep = chunk.root.head.dep_
  chunk_root_head_lemma = chunk.root.head.lemma_
  chunk_ents = chunk.ents
  chunk_root_ent_type = chunk.root.ent_type_
  
  print(f'1.chunk.text: {chunk_text:>15} | 2.ch.root: {chunk_root:>14} | 3.chunk.root.dep_: {chunk_root_dep:>18} | 4.ch.root.head: {chunk_root_head:>12} | 5.ch.root.head.dep_: {chunk_root_head_dep:>10} |  6.chunk.root.head.lemma_: {chunk_root_head_lemma:>9}')

print()
chunks = list(doc.noun_chunks)
for i in range(len(chunks)):
    print(f'idx: {i} | {chunks[i].text:>13} | {chunks[i].start:>2} {chunks[i].end:>2} | {chunks[i].start_char:>2}  {chunks[i].end_char:>2} || {chunks[i].root.ent_type_:>10} {chunks[i].ents} ')

print()
# Lemmatization for tokens 
lemmatization = pd.DataFrame(data=[], \
  columns=["id", "Texto","Lemma", "Tag", "Tag_explainned", "token_POS", "POS_explainned", "dep", "T. Head", "dep explained"])
i = 0
for token in doc:
    lemmatization.loc[i,"id"] = token.i
    lemmatization.loc[i,"Texto"] = token.text
    lemmatization.loc[i,"Lemma"] = token.lemma_
    lemmatization.loc[i,"Tag"] = token.tag_
    lemmatization.loc[i,"Tag_explainned"] = spacy.explain(token.tag_)
    lemmatization.loc[i,"token_POS"] = token.pos_
    lemmatization.loc[i,"POS_explainned"] = spacy.explain(token.pos_)
    lemmatization.loc[i,"dep"] = token.dep_
    lemmatization.loc[i,"T. Head"] = token.head.text
    lemmatization.loc[i,"dep explained"] = token.morph
    
    i = i+1
lemmatization

In [ ]:
def convert_resize_gray(original_file_name, file_path, image_resized_path):

    name_image = utl.conv_filename_no_ext(original_file_name)
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    pages = convert_from_path(file_path, 500, poppler_path=poppler_path)
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((4134, 5846))
            resized_pages.append(resized_page)
            
    imagem_gray = resized_pages[0].convert('L')
    imagem_gray.save(image_resized_name, 'JPEG')

    return  imagem_gray, image_resized_name

In [ ]:
pattern_valortotal2 = {
    "label": "TOTALNOTA",
    "pattern": [
        {"LOWER": "valor"},
        {"IS_SPACE": True, "OP": "*"},
        {"LOWER": "total"},
        {"IS_SPACE": True, "OP": "*"},
        {"LOWER": "da"},
        {"IS_SPACE": True, "OP": "*"},
        {"LOWER": "nota"},
        {"IS_SPACE": True, "OP": "*"},
        {"TEXT": {"REGEX": "R\$"}},
        {"IS_SPACE": True, "OP": "*"},
        {"TEXT": {"REGEX": "[0-9]{1,3}(\.[0-9]{3})*(,[0-9]{2})?"}, "OP": "?"}
    ]
}





patternsOthers = [{"label": "PERSON", "pattern": "Daniel", "id": "pessoa-daniel"},
                  {"label": "PERSON", "pattern": "Antônio", "id": "pessoa-antonio"}] 
 
patternsCult = [
    {
        "label":"CULTURA",
        "pattern": [
            {"LOWER": "soja", "OP":"?"},
            {"LOWER": "milho", "OP":"?"},
            {"LOWER": "sorgo", "OP":"?"},
            {"LOWER": "trigo", "OP":"?"},
            {"LOWER": "milheto", "OP":"?"},  
            
        ],    
    "id": "cultura"}]

patternsValores = [{"label":"VALOR_SERVICO", "pattern": [{"LOWER": "valor", "OP":"?"}, {"LOWER": "serviços", "OP":"*"}], "id": "val-servico"},
                 {"label":"ENTREGUE", "pattern": [{"LOWER": "quantidade", "OP":"?"},{"LOWER": "entregue","OP":"*"}],"id": "qtde-entregue"},
                 {"label":"ENTREGUE", "pattern": [{"LOWER": "quantidade", "OP":"?"},{"LOWER": "entreguei","OP":"*"}],"id": "qtde-entregue"},
                 {"label":"ENTREGUE", "pattern": [{"LOWER": "foram", "OP":"?"},{"LOWER": "entregues","OP":"*"}],"id": "qtde-entregue"},
                 {"label":"SALDO", "pattern": [{"LOWER": "saldo","OP":"*"}], "id": "qtde-saldo"}]


patternsSafra = [{"label":"SAFRA", "pattern": [{"LOWER": "safra", "OP":"?"},{"LOWER": "safras", "OP":"?"}], "id": "safra"}]


patternsNroSafra = [{"label":"NR_SAF", "pattern": [{"SHAPE": "dd/dd", "OP":"*"}], "id": "nro_safra"},
                    {"label":"NR_SAF", "pattern": [{"lower": "próxima", "OP":"*"}], "id": "nro_safra"},
                    {"label":"NR_SAF", "pattern": [{"lower": "passada", "OP":"*"}], "id": "nro_safra"}]   

patternsCliente = [{"label": "CLIENTE", "pattern": [{"LOWER": "berdinazzi"}], "id": "cli-berdinazzi"},
                   {"label": "CLIENTE", "pattern": [{"LOWER": "lopito"}], "id": "cli-lopito"},
                   {"label": "CLIENTE", "pattern": [{"LOWER": "bungue"}], "id": "cli-bungue"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["bunge", "bongue", "bumgue"]}}}], "id": "cli-bungue"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["berdinazi"]}}}], "id": "cli-berdinazzi"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["matarazzo"]}}}], "id": "cli-matarazzo"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["mezenga"]}}}], "id": "cli-mezenga"},
                   {"label": "CLIENTE", "pattern": [{"LOWER": "rei"}, {"LOWER": "do"}, {"LOWER": "gado"}], "id": "cli-reidogado"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["rei-do-gado"]}}}], "id": "cli-reidogado"},   
                   ]

patternsOthersFazenda = [{"label": "FAZENDA", "pattern": [{"LOWER": "santa"}, {"LOWER": "rita"}], "id": "faz-santarita"},
                         {"label": "FAZENDA", "pattern": [{"LOWER": "bela"}, {"LOWER": "vista"}], "id": "faz-belavista"},
                         {"label": "FAZENDA", "pattern": [{"LOWER": "passo"}, {"LOWER": "fundo"}], "id": "faz-passo"},
                         {"label": "FAZENDA", "pattern": [{"LOWER": "minha"}, {"LOWER": "fazenda"}], "id": "faz-produtor"},
                        {"label": "FAZENDA", "pattern": [{"LOWER": "agrobi"}], "id": "faz-produtor"}
                        ]
 
patternsContrato = [{"label": "CONTRATO", "pattern": [{"LOWER": "contrato", "OP":"?"}], "id": "contrato"},
                    {"label": "CONTRATO", "pattern": [{"LOWER": "contratos", "OP":"?"}], "id": "contrato"}]

patternsNroContrato = [{"label": "NR_CONT", "pattern": [{"SHAPE": "dddX", "OP":"*"}], "id": "nro_contrato"},
                       {"label": "NR_CONT", "pattern": [{"POS": "NUM", "SHAPE": "ddd", "OP": "*"},
                                                            {"POS": "PROPN", "SHAPE": "X", "OP": "*"}], "id": "nro_contrato"}]

patternsIntent = [{"label": "INTENT", "pattern": [{"IS_TITLE": True, "OP":"*"}], "id": "user-intent"},
                  {"label": "INTENT", "pattern": [{"POS": "ADJ", "OP":"*"}, {"POS": "VERB", "OP":"*"}], "id": "user-intent"},
                  {"label": "INTENT", "pattern": [{"TEXT": {"FUZZY": {"IN": ["preciso", "gostaria", "informar"]}}}], "id": "user-intent"}]


In [ ]:

colors = {
        "CULTURA": "linear-gradient(90deg, #2ADB5E, #1FA346)", 
        "TOTAL": "linear-gradient(90deg, #09D6FF, #08A0D1)",
        "ENTREGUE": "linear-gradient(90deg, #09D6FF, #08A0D1)",
        "SALDO": "linear-gradient(90deg, #09D6FF, #08A0D1)", 
        "FAZENDA": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", 
        "SAFRA": "linear-gradient(90deg, #FFC90E, #BA930A)", 
        "CONTRATO": "linear-gradient(90deg, #B5B5B5, #8A8A8A)",
        "INTENT": "linear-gradient(90deg, #09D6FF, #08A0D1)"}


patternsOthers = [{"label": "PERSON", "pattern": "Daniel", "id": "daniel"},
                  {"label": "PERSON", "pattern": "Antônio", "id": "antônio"},
                  {"label": "ORG", "pattern": [{"LOWER": "fast"}, {"LOWER": "innovation"}], "id": "fast-innovation"},
                  {"label": "ORG", "pattern": {"LOWER": "agrobi"}, "id": "agrobi"},
                  {"label": "ORG", "pattern": {"LOWER": "AgroBi"}, "id": "agrobi"},
                  {"label": "ORG", "pattern": [{"LOWER": "agro"}, {"LOWER": "bi"}], "id": "agrobi"},
                  ] 
 

patternsCult = [
    {
        "label":"CULTURA",
        "pattern": [
            {"LOWER": "soja", "OP":"?"},
            {"LOWER": "milho", "OP":"?"},
            {"LOWER": "sorgo", "OP":"?"},
            {"LOWER": "trigo", "OP":"?"},
            
        ]    
    }
]


patternsQuant = [
    {
        "label":"TOTAL",
        "pattern": [
            {"LOWER": "total"},
            {"LOWER": "quantidade", "OP":"?"},
            
        ]    
    },
    {
        "label":"ENTREGUE",
        "pattern": [
            {"LOWER": "quantidade", "OP":"?"},
            {"LOWER": "entregue","OP":"?"},
            {"LOWER": "entregues","OP":"?"},
            {"LOWER": "entregado","OP":"?"},
            {"LOWER": "entreguei","OP":"?"},
        ]    
    },
    {
        "label":"SALDO",
        "pattern": [
            {"LOWER": "saldo"},
            {"LOWER": "quantidade", "OP":"?"},
            {"LOWER": "total", "OP":"?"},
        ]    
    }
    
]

patternsSafra = [
    {
        "label":"SAFRA",
        "pattern": [
            {"LOWER": "safra", "OP":"?"},
            {"LOWER": "safras", "OP":"?"},
            {"SHAPE": "dd/dd"},
        ]    
    }
]

patternsOthersFazenda = [{"label": "FAZENDA", "pattern": [{"LOWER": "santa"}, {"LOWER": "rita"}], "id": "faz-santa-rita"},
                         {"label": "FAZENDA", "pattern": [{"LOWER": "bela"}, {"LOWER": "vista"}], "id": "faz-bela-vista"},
                         {"label": "FAZENDA", "pattern": [{"LOWER": "passo"}, {"LOWER": "fundo"}], "id": "faz-passo-fundo"},
                         {"label": "FAZENDA", "pattern": [{"LOWER": "minha"}, {"LOWER": "fazenda"}], "id": "faz-produtor"}
                        ]
 

patternsContrato = [
    {
        "label":"CONTRATO",
        "pattern": [
            {"LOWER": "contrato", "OP":"*"},
            {"SHAPE": "dddX", "OP":"*"},
            {"LOWER": "contratos", "OP":"*"},
            {"SHAPE": "dddX", "OP":"*"}
            
        ]
        
    }
    
]







patterns = patternsCult + patternsQuant + patternsOthersFazenda + patternsSafra + patternsContrato + patternsOthers

In [ ]:
patterns

In [ ]:
nlp.add_pipe("merge_noun_chunks")

In [ ]:
imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)

In [ ]:
ocr_text = (pytesseract.image_to_string(imagem_gray, lang='por'))
ocr_text

In [ ]:
# Principais funcoes

def busca_emitente(remoteJid):
    
    for entidade in entidades.find({"remoteJid": remoteJid}):
            nome_entidade = entidade['Nome']
            sobrenome_entidade = entidade['Sobrenome']
            tipo_entidade = entidade['Tipo']
            razao_entidade = entidade['razao']
            
    return  nome_entidade, sobrenome_entidade, tipo_entidade, razao_entidade


def show_ent_new(text, patterns):
    #nlp = spacy.blank("pt")
    #ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)
    doc = nlp(text)
    
    tokens = []
    ents = []
    
    for ent in doc.ents:
        span = doc.char_span(ent.start_char, ent.end_char, label=ent.label_)
        ents.append(span)
        
    for token in doc:
        start = token.idx
        end = start + len(token)
        tokens.append((token.text, start, end))
        
    return doc, tokens, ents



# chunk.text, chunk.start, chunk.end, chunk.root.head.lemma_, chunk.root.dep_, chunk.doc
def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

In [ ]:
colors = {
        "TOTALNOTA": "linear-gradient(90deg, #2ADB5E, #1FA346)", 
        "TOTAL": "linear-gradient(90deg, #09D6FF, #08A0D1)",
        "ENTREGUE": "linear-gradient(90deg, #09D6FF, #08A0D1)",
        "SALDO": "linear-gradient(90deg, #09D6FF, #08A0D1)", 
        "FAZENDA": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", 
        "SAFRA": "#FFEA7F",
        "NR_SAF": "#CCA10C", 
        "CONTRATO": "#AB9BFC",
        "NR_CONT": "#7AECEC",
        "INTENT": "#EE8AF8",
        "INTENT": "linear-gradient(90deg, #09D6FF, #08A0D1)",
        "PERSON": "linear-gradient(90deg, #FFA9FB, #BF7FBC)",
        "CLIENTE": "linear-gradient(90deg, #FFA9FB, #BF7FBC)"}



pattern_valortotal = [{
    "label": "TOTALNOTA",
    "pattern": [
        {"LOWER": "valor"},
        {"IS_SPACE": True, "OP": "*"},
        {"LOWER": "total"},
        {"IS_SPACE": True, "OP": "*"},
        {"LOWER": "da"},
        {"IS_SPACE": True, "OP": "*"},
        {"LOWER": "nota"},
        {"IS_SPACE": True, "OP": "*"},
        {"TEXT": {"REGEX": "R\$"}},
        {"IS_SPACE": True, "OP": "*"},
        {"TEXT": {"REGEX": "[0-9]{1,3}(\.[0-9]{3})*(,[0-9]{2})?"}, "OP": "?"}
    ]
}]




patterns = pattern_valortotal

In [ ]:
# 1. XXX Executar o comando OCRmyPDF
!ocrmypdf --language por --deskew 'pipeline_extracao_documentos/0_arquivos_teste_pipeline/pdf_raster/Mage/Doria Marinho 0295 Carlos Leandro.pdf' 'output.pdf'

In [ ]:
!ocrmypdf --language por --deskew '/home/dani-boy/extractNF/pipeline_extracao_documentos/0_arquivos_teste_pipeline/pdf_raster/Sao_predo_aldeia/2023143.pdf' 'output.pdf'